## GK based MCQ Generator

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [26]:
import openai
import json
import time

from prompts import SIMPLE_GK_TEXT_MCQ_PROMPT
from core.services.qna_gen.utils import add_dicts

from dotenv import load_dotenv
load_dotenv()

True

### Text MCQs

In [4]:
context = """
INDIA
National Conference-Congress alliance sweeps Kargil election with 22 of 26 seats; BJP wins two.
The Chief of the Air Staff unveiled a new Ensign for the force, as it marked its 91st anniversary.
The Army has completed the fourth tranche of Emergency Procurements (EP).
The Indian Space Research Organisation (ISRO) has performed a trajectory correction manoeuvre on the Aditya-L1 spacecraft.
22 people drowned during Jivitputrika festival in Bihar.
President of India Droupadi Murmu inaugurated an International Research Conference on ‘From research to impact: Towards Just and Resilient Agri-Food Systems’.
The 10th ” India-Sweden Innovation Day” Meet was held in Sweden.
National Current Good Manufacturing Practice Day (cGMP Day) is to be celebrated on 10 October.
Balasore train accident: Bhubaneswar Municipal Corporation to scientifically dispose 28 unclaimed bodies.
ECONOMY & CORPORATE
Automotive retail in September 2023 showcased a robust 20 percent double-digit year on year growth.
Over 3 million demat accounts opened in Sept for second consecutive month.
WORLD
Israeli air strikes attacks Gaza killing more than 370 people, including 20 children.
World Post Day is celebrated each year on 9 October.
Bahrain, Iraq, Kuwait, Oman, Saudi Arabia and UAE reaffirmed their commitment to voluntary adjustments” to oil production.
President Joe Biden ordered U.S. ships and warplanes to move closer to Israel in a show of support.
SPORTS
19th Asian Games concludes in Hangzhou, China while India is ranked at fourth place.
For the first time in 60 years, India has won 107 medals in Asian Games, including 28 gold.
"""

In [27]:

def generate_text_mcqs(context, n=1, example = None, model_id = "gpt-4-0613"):

    system_message = {"role": "system", "content": SIMPLE_GK_TEXT_MCQ_PROMPT}
    conversation = [system_message]

    global total_usage
    total_usage = {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0}

    user_prompt = f"""
    CONTEXT_TEXT:
    //context//

    {context}

    //context//
   
    N: {n}

    EXAMPLE:
    {example if example else "NA"}

    QUESTIONS: 
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    response = openai.ChatCompletion.create(
                                    model= model_id,
                                    messages = conversation,
                                    temperature = 1
                                    )
    
    total_usage = add_dicts(total_usage, dict(response.usage))

    output = response.choices[0].message.content

    return {"output": output, "total_usage": total_usage}




In [ ]:
example = {
        "question": "Which of the following statements is true regarding thermodynamics?",
        "choices": ["The surroundings refer to the part of the universe under study.", "State Functions depend on the path taken during a change.", "In an adiabatic process, there is no heat exchange with the surroundings.", "The First Law of Thermodynamics is represented by the equation ΔU = q - w."],
        "answer": "In an adiabatic process, there is no heat exchange with the surroundings."
    }

# output = generate_text_mcqs(
#                         context = context,
#                         n = 5,
#                         # example = example,
#                         )

# print(output[0])
# print(output[1])

In [28]:
path = r"C:\Users\DELL\Documents\Curate\curate-v1\core\services\qna_gen\gk\test_data.txt"

with open(path, 'rb') as f:
    text = f.read().decode('utf-8')

text = text.replace('\r', '')
splitted_text = text.split('\n\n\n')

# text

In [ ]:
# batch qna gen

questions = []
left = []

for context_split in splitted_text:

    result = generate_text_mcqs(
                        context = context_split,
                        n = 3,
                    )
    print(result)

    try:
        json_ques = json.loads(result['output'])
        questions.extend(json_ques['mcqs'])
        print(json_ques['mcqs'])
        print()

    except:
        left.append(context_split)

        


### Numerical MCQs

In [13]:
functions = [
    {
      "name": "get_results_from_wolfram",
      "description": "This function will return a list of accurate calculation results for the mathematical queries",
      "parameters": {
        "type": "object",
        "properties": {
          "queries": {
            "type": "string",
            "description": """the list of mathematical expressions for example - "['2^2 + 3', 'ln(5) + 2*4']". ONLY mathematical expressions are supported."""
          }
        }
      },
        "required": ["queries"]
    }
] 